<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import math
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler 

In [44]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)
def sigmoid(output):
  return [1.0/(1.0+np.exp(-z)) for z in output]
def relu(x):
  output = [0] * len(x)
  for index, item in enumerate(x): 
    output[index] = item if item > 0.0 else item * 0.05
  return output

In [3]:
def categorical_cross_entropy(actual, predicted):
	sum_score = 0.0
	for i in range(len(actual)):
			sum_score += actual[i] * math.log(1e-15 + predicted[i])
	mean_sum_score = (1.0 / len(actual)) * sum_score
	return -mean_sum_score

In [4]:
output_labels = [0.0, 1.0, 2.0]

In [45]:
'''
Class for the layer of the network 
which manages weights, bias, output and errors  
'''
class Layer():
  def __init__(self, name, units, input_dim):
    self.name = name
    self.w = [[random.uniform(0,1) for _ in range(input_dim)] for _ in range(units)]
    self.b = [0 for _ in range(units)]
    self.output = [bias for bias in self.b]
    self.next_layer = None
    self.previous_layer = None
    self.error = [0 for _ in range(units)]
  def set_previous_layer(self, previous_layer):
    self.previous_layer = previous_layer
  def set_next_layer(self, next_layer):
    self.next_layer = next_layer
  '''
  function to calculate the output of the neurons at current layer with 
  the given inputs
  '''
  def calculate_output(self, inputs, activation=sigmoid):
    self.output = [bias for bias in self.b]
    self.output += np.dot(self.w, inputs)
    self.output = activation(self.output)
    return self.output 
  '''
  function to calculate the error in the output at current layer with 
  the given expected output
  '''
  def update_error(self, expected_output):
    # last layer
    if self.next_layer == None:
      inputs = self.previous_layer.output
      # total_error = sum([(exp_output-self.output[i])**2 for i, exp_output in enumerate(expected_output)])
      # total_error = log_loss(expected_output, [self.output], labels=output_labels)
      total_error = categorical_cross_entropy(self.output, expected_output)
      for row_index, col in enumerate(self.w):
        self.error[row_index] = total_error * self.output[row_index] * (1-self.output[row_index]) 
    # layers other than last layer
    else:
      weights = self.next_layer.w
      next_layer_error = self.next_layer.error
      for output_idx, output in enumerate(self.output):
        self.error[output_idx] = sum([weight[output_idx] * next_layer_error[weight_idx] * output * (1-output) for weight_idx, weight in enumerate(weights)])
  '''
  function to update the weights at current layer with 
  the given inputs
  '''
  def update_weights(self, learning_rate, row):
    if self.next_layer == None:
      inputs = self.previous_layer.output
      for row_index, col in enumerate(self.w):
        for col_index, weight in enumerate(col):
          self.w[row_index][col_index] = self.w[row_index][col_index] + (learning_rate * self.error[row_index] * inputs[col_index])
          self.b[row_index] += learning_rate * self.error[row_index]
    else:
      inputs = row
      for row_index, col in enumerate(self.w):
        for col_index, weight in enumerate(col):
          self.w[row_index][col_index] = self.w[row_index][col_index] + (learning_rate * self.error[row_index] * inputs[col_index])
          self.b[row_index] += learning_rate * self.error[row_index]
  
    

In [6]:
# create network with the hidden and output layers
def reset_layers(inputs, hidden_layer, output):
  layer2 = Layer("Output", output, hidden_layer)
  layer1 = Layer("Hidden1", hidden_layer, inputs)
  layer2.set_previous_layer(layer1)
  layer1.set_next_layer(layer2)
  layers = [layer1, layer2]
  return layers

In [7]:
# Load IRIS data
iris = load_iris()

iris_columns = ['sepal_len', 'sepal_width', 'petal_length', 'petal_width', 'target']
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris_columns)

In [47]:
# select desired columns, split into training and test datasets
# standardize the inputs
from sklearn.model_selection import train_test_split
# X = df[['sepal_len', 'sepal_width', 'petal_length', 'petal_width']]
X = df[['sepal_width', 'petal_length']]
y = df[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state = 10)


scaler_training = StandardScaler() 
scaler_training.fit(X_train)
scaled_values_training = scaler_training.transform(X_train)
scaled_values_testing = scaler_training.transform(X_test)

X_train = pd.DataFrame(scaled_values_training, columns=[*X_train.columns.values])
X_test = pd.DataFrame(scaled_values_testing, columns=[*X_test.columns.values])

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


In [46]:
# set random seed
import random
random.seed(68)
selected_rows = random.sample(range(0,119),   5)

In [10]:
# label binarizer
lb = preprocessing.LabelBinarizer()
lb.fit([0,1,2])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [48]:
def train_weights(X_train, y_train, num_epochs, learning_rate, layers):
  index = 0;
  # select mini batch
  mini_batch = X_train.loc[selected_rows, :]
  y_train = y_train.loc[selected_rows, :]
  while index < num_epochs:
    sum_error = 0
    print("============================================")
    print("=================Iteration %d==============="%(index+1))
    print("============================================")
    for rowIndex, row in mini_batch.iterrows():
      # pass the inputs and get the outputs in a typical feed forward network.
      output = row
      for layer_idx, layer in enumerate(layers):
        if layer_idx == len(layers) - 1:
          output = layer.calculate_output(output, softmax)
        else:
          output = layer.calculate_output(output, sigmoid)
      # calculate total error 
      expected_output = lb.transform([y_train.loc[rowIndex,'target']])[0]
      sum_error += sum([(exp_output-output[i])**2 for i, exp_output in enumerate(expected_output)])

      # update error at each layer
      for layer_idx, layer in enumerate(reversed(layers)):
        # layer.update_error([y_train.loc[rowIndex,'target']])
        layer.update_error(expected_output)
      print("error of output layer", layers[1].error)
      print("error of hidden layer", layers[0].error)  
      # update weights at each layer
      for layer_idx, layer in enumerate(reversed(layers)):
        layer.update_weights(learning_rate, row)
      
      # error = log_loss([y_train.loc[rowIndex,'target']], [output], labels=[0.0, 1.0, 2.0])
    print("Entropy based loss: %f"% (sum_error))
      # if error != 0:
      #   layer1_error = layers[1].update_weights(error, layers[0].output, learning_rate)
      #   layers[0].update_weights(layer1_error, row, learning_rate)
    print("Weights of output layer", layers[1].w)
    print("Weights of hidden layer", layers[0].w)
    index = index + 1

layers = reset_layers(2, 3, 3)
print("Creating model using training data")
train_weights(X_train, y_train, 2, 0.5, layers)

Creating model using training data
=================Iteration 1===============
error of output layer [1.7368157915950448, 1.6767918348107558, 1.7056146479985879]
error of hidden layer [0.6942278677700293, 0.5730120695440293, 0.7657205268975965]
error of output layer [1.8302220188740579, 1.6428293788645632, 1.7042421855178111]
error of hidden layer [0.828468892453033, 1.0340210187181174, 1.105484380183395]
error of output layer [1.9884675466869537, 1.4681775440283094, 1.695084394989339]
error of hidden layer [1.1124230745194597, 1.3690902880309834, 1.174936691979572]
error of output layer [1.0542447172974283, 0.45675343678025687, 0.8766261243918505]
error of hidden layer [1.5411237956848653, 0.8275064499661879, 0.9730990758692253]
error of output layer [1.7351031152199201, 0.2488981079514728, 1.571695262110323]
error of hidden layer [0.0651524343309799, 0.08310206865349759, 0.07678088919461337]
Entropy based loss: 3.984325
Weights of output layer [[4.156781889633132, 3.099426532442715, 

In [39]:
# select desired columns, split into training and test datasets
# standardize the inputs
from sklearn.model_selection import train_test_split
X = df[['sepal_len', 'sepal_width', 'petal_length', 'petal_width']]
y = df[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state = 10)


scaler_training = StandardScaler() 
scaler_training.fit(X_train)
scaled_values_training = scaler_training.transform(X_train)
scaled_values_testing = scaler_training.transform(X_test)

X_train = pd.DataFrame(scaled_values_training, columns=[*X_train.columns.values])
X_test = pd.DataFrame(scaled_values_testing, columns=[*X_test.columns.values])

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


In [40]:
# set random seed
import random
random.seed(68)
selected_rows = random.sample(range(0,119), 10)

In [43]:
layers = reset_layers(4, 5, 3)
train_weights(X_train, y_train, 2, 0.5, layers)

=================Iteration 1===============
error of output layer [1.0786260194400918, 1.2831662785205549, 1.5200857289182272]
error of hidden layer [0.43443858452978223, 0.41460107095467885, 0.3389054332104148, 0.5265562539556266, 0.13913829338856626]
error of output layer [0.1367448163016792, 0.3970587082474105, 0.4817887444450306]
error of hidden layer [0.09301873563781757, 0.0675850030477192, 0.05825835800504621, 0.06479257903013767, 0.09328441175521114]
error of output layer [0.03055085002042671, 0.24357555825980234, 0.2648197988593317]
error of hidden layer [0.1261248288709685, 0.0853724660882837, 0.09749618441777483, 0.13671432559041924, 0.10467650996252573]
error of output layer [0.49433410624364826, 2.213802378632092, 2.4170038899349837]
error of hidden layer [0.7469677654062885, 0.30214602045469663, 0.1348877567495412, 0.4051506783034388, 0.42078433584048336]
error of output layer [0.00031790403248639865, 0.7791763099539799, 0.7794433783706572]
error of hidden layer [0.146241